In [1]:
import pandas as pd
from nsepy import get_history as gh
import datetime as dt
import yfinance as yf
import datetime as dt

In [2]:
start = dt.datetime(2021, 7, 1)
end = dt.datetime(2022, 1, 27)
symbol = 'AXISBANK.NS'  # Stock symbol in uppercase

stk_data = yf.download(symbol, start=start, end=end)

[*********************100%%**********************]  1 of 1 completed


In [3]:
stk_data=stk_data[["Open","High","Low","Close"]]

In [4]:
#column="Close"

from sklearn.preprocessing import MinMaxScaler
Ms = MinMaxScaler()
data1= Ms.fit_transform(stk_data)
print("Len:",data1.shape)

Len: (143, 4)


In [5]:
data1=pd.DataFrame(data1,columns=["Open","High","Low","Close"])
dataset=data1

In [6]:
training_size = round(len(data1 ) * 0.80)
print(training_size)
X_train=data1[:training_size]
X_test=data1[training_size:]
print("X_train length:",X_train.shape)
print("X_test length:",X_test.shape)
y_train=data1[:training_size]
y_test=data1[training_size:]
print("y_train length:",y_train.shape)
print("y_test length:",y_test.shape)

114
X_train length: (114, 4)
X_test length: (29, 4)
y_train length: (114, 4)
y_test length: (29, 4)


In [7]:
import warnings
warnings.filterwarnings("ignore")

In [8]:
performance={"Model":[],"RMSE":[],"MaPe":[],"Lag":[],"Test":[]}

In [9]:
def combination(dataset, listt):
    """
    Function to perform VARMA model parameter tuning and forecasting.

    Parameters:
    - dataset: Pandas DataFrame or Series containing the time series data.
    - listt: List of variables to use for forecasting.

    Returns:
    - perf: DataFrame containing performance metrics.
    - result: Fitted VARMA model.
    - pred: Forecasted values.
    """
    print(listt)
    datasetTwo = dataset[listt]
    test_obs = 28
    train = datasetTwo[:-test_obs]
    test = datasetTwo[-test_obs:]
    
    from statsmodels.tsa.statespace.varmax import VARMAX
    from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error

    performance = {
        "Model": [],
        "RMSE": [],
        "MaPe": [],
        "Lag": [],
        "Test": []
    }

    best_aic = float('inf')
    best_params = None

    for p in range(1, 4):  # AR order
        for q in range(1, 4):  # MA order
            try:
                model = VARMAX(train, order=(p, 0, q))
                results = model.fit(maxiter=1000, disp=False)
                aic = results.aic
                if aic < best_aic:
                    best_aic = aic
                    best_params = (p, 0, q)
            except:
                continue

    print("Best AIC:", best_aic)
    print("Best Parameters (p, d, q):", best_params)

    # Fit VARMA model with the best parameters
    model = VARMAX(train, order=best_params) 
    results = model.fit()

    # Forecast
    pred = results.forecast(steps=test_obs)

    # Calculate performance metrics
    rmse = round(mean_squared_error(test, pred, squared=False))
    mape = mean_absolute_percentage_error(test, pred)
    
    # Store performance metrics
    performance["Model"].append(listt)
    performance["RMSE"].append(rmse)
    performance["MaPe"].append(mape)
    performance["Lag"].append(best_params)
    performance["Test"].append(test_obs)
    
    perf = pd.DataFrame(performance)
    return perf, results, pred

In [10]:
listt=["Close","High","Open","Low"]
#listt=["AQI_calculated","PM10","PM2.5","NOx","NO2","NO","NH3","SO2","CO",'year']

In [11]:
perf,result,pred=combination(data1, listt)

['Close', 'High', 'Open', 'Low']
Best AIC: -1918.7183228483696
Best Parameters (p, d, q): (2, 0, 1)


In [12]:
data1

,Open,High,Low,Close
0,0.508982,0.441549,0.543815,0.491495
1,0.497006,0.433854,0.540997,0.513145
2,0.544411,0.479275,0.593970,0.563144
3,0.543912,0.504095,0.604114,0.552835
4,0.538423,0.489203,0.587489,0.576289
...,...,...,...,...
138,0.385230,0.330107,0.398422,0.404639
139,0.414421,0.355920,0.417864,0.401031
140,0.369261,0.296848,0.338405,0.316495
141,0.334331,0.252916,0.226543,0.275773


In [13]:
perf

,Model,RMSE,MaPe,Lag,Test
0,"[Close, High, Open, Low]",0,1.256465,"(2, 0, 1)",28
